# final project - KICKSTARTER

**Introduction**

bla bla bla bla bla 

In [121]:
from textblob import TextBlob

In [122]:
a = TextBlob("was soll ich machen?")
a.detect_language()

'de'

# testing NLP - natural language processing

In [81]:
import yake

In [82]:
kw_extractor = yake.KeywordExtractor()

In [113]:
text = """Nicole, es ist natürlich sehr schade für mich, dass Du nicht dabei sein kannst. Dennoch brauchst Du Dich nicht bei mir entschuldigen, denn es ist ja auch vor allem Schade für Dich, dass Du bei unserem schönen Treffen nicht dabei sein kannst. 
Warum lässt Du Joel nicht alleine mit dem Lütten nach Spanien fahren und kommst zu uns? Ich meine Du hast so viel für die Familie getan, jetzt kann die Familie auch was für Dich tun!
Denn gerade in so turbulenten braucht man erst Recht eine Auszeit und Joel sollte Dich darin bestimmt gut supporten können. """

In [92]:
language = "de"

In [107]:
max_ngram_size = 1

In [108]:
deduplication_threshold = 0.9

In [109]:
numOfKeywords = 5

In [114]:
custom_kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_threshold, top=numOfKeywords, features=None)

In [115]:
keywords = custom_kw_extractor.extract_keywords(text)

In [116]:
for kw in keywords:
    print(kw)

('schade', 0.08811238611117093)
('Nicole', 0.09696306862068063)
('Familie', 0.1512733699895294)
('Joel', 0.1638096544732938)
('Treffen', 0.2195416409690374)


## import libraries

In [48]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import glob

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import TomekLinks

import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy import stats
from scipy.stats.mstats import winsorize

import math
from sklearn.preprocessing import OneHotEncoder, Normalizer, LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import mean_squared_error, r2_score, confusion_matrix

from sklearn.neighbors import KNeighborsRegressor

import warnings
warnings.filterwarnings('ignore')

## import data

In [53]:
files = os.listdir('Data')
files

['kickstarter_all_enddate_30102021.xlsx',
 'kickstarter_all_mostbacked.xlsx',
 'kickstarter_all_newest_30102021.xlsx',
 'kickstarter_all_popularity_30102021.xlsx',
 'kickstarter_art_enddate.xlsx',
 'kickstarter_art_newest.xlsx',
 'kickstarter_comics_enddate.xlsx',
 'kickstarter_comics_newest.xlsx',
 'kickstarter_crafts_enddate.xlsx',
 'kickstarter_crafts_newest.xlsx',
 'kickstarter_dance_enddate.xlsx',
 'kickstarter_dance_enddate2.xlsx',
 'kickstarter_dance_newest.xlsx',
 'kickstarter_design_enddate.xlsx',
 'kickstarter_design_newest.xlsx',
 'kickstarter_fashion_enddate.xlsx',
 'kickstarter_fashion_newest.xlsx',
 'kickstarter_film_enddate.xlsx',
 'kickstarter_food_enddate.xlsx']

In [54]:
files = os.listdir('Data')

df = pd.DataFrame()

path = os.getcwd() + "\\Data" + "\\"

for i in files:
    data = pd.read_excel(path + i)
    df = df.append(data)

In [19]:
#data = pd.read_excel(r'Data\kickstarter_art_newest.xlsx')

## first review and clean data

In [56]:
df.shape

(17304, 47)

In [58]:
df.sample(5)

,backers_count,blurb,categoryId,categoryName,categorySlug,converted_pledged_amount,country,country_displayable_name,created_at,created_at_formatted,...,state,state_changed_at,static_usd_rate,title,url,usd_exchange_rate,usd_pledged,usd_type,is_backing,is_starred
1541,23,Cosmically cute enamel pins brought to you by ...,26,Crafts,crafts,428,US,the United States,1488397993,"Wed, 01 Mar 2017 19:53:13 +0000",...,successful,1490619335,1.000000,Sugar Lagoon: Space Cat Enamel Pins,https://www.kickstarter.com/projects/sugarlago...,1.00000,428.000000,domestic,NaN,NaN
307,278,"Magnet Phone Stand can hold 5+ cards, cash, co...",28,Product Design,design/product design,9125,US,the United States,1631499876,"Mon, 13 Sep 2021 02:24:36 +0000",...,live,1633060044,1.000000,Magnetic Phone Wallet/Stand for MagSafe,https://www.kickstarter.com/projects/elusefor/...,1.00000,9125.000000,international,NaN,NaN
906,8,We will be creating a line of Soaps inspired b...,345,DIY,crafts/diy,285,US,the United States,1543680581,"Sat, 01 Dec 2018 16:09:41 +0000",...,successful,1548262482,1.000000,Fantasy Soaps by FireBunny Games,https://www.kickstarter.com/projects/cymultech...,1.00000,285.000000,international,NaN,NaN
192,39,Wir werden im Herbst 2019 unsere nächste Kreat...,254,Performances,dance/performances,5743,DE,Germany,1546505894,"Thu, 03 Jan 2019 08:58:14 +0000",...,successful,1548586801,1.155864,Romeo ∅ Julia - eine Performance für Jugendliche,https://www.kickstarter.com/projects/amadeuspa...,1.14065,5819.773125,international,NaN,NaN
38,27,"When a Volcano suddenly erupts, a team is sent...",3,Comics,comics,833,US,the United States,1633276132,"Sun, 03 Oct 2021 15:48:52 +0000",...,live,1635221675,1.000000,ECLIPSE: THE HAWAIIAN PROTECTOR is a WANTED MAN,https://www.kickstarter.com/projects/continuum...,1.00000,833.000000,international,NaN,NaN


### Cleaning Column Names

In [68]:
df.columns

Index(['backers_count', 'blurb', 'categoryId', 'categoryName', 'categorySlug',
       'converted_pledged_amount', 'country', 'country_displayable_name',
       'created_at', 'created_at_formatted', 'creatorAvatar', 'creatorId',
       'creatorName', 'creatorUrl', 'currency', 'currency_symbol',
       'currency_trailing_code', 'current_currency', 'deadline', 'description',
       'disable_communication', 'fx_rate', 'goal', 'id', 'is_starrable',
       'launched_at', 'launched_at_formatted', 'link', 'locationId',
       'locationName', 'name', 'photo', 'pledged', 'pubDate', 'slug',
       'spotlight', 'staff_pick', 'state', 'state_changed_at',
       'static_usd_rate', 'title', 'url', 'usd_exchange_rate', 'usd_pledged',
       'usd_type', 'is_backing', 'is_starred'],
      dtype='object')

no cleaning of column names required

### Review DTypes

In [69]:
df.dtypes

backers_count                 int64
blurb                        object
categoryId                    int64
categoryName                 object
categorySlug                 object
converted_pledged_amount      int64
country                      object
country_displayable_name     object
created_at                    int64
created_at_formatted         object
creatorAvatar                object
creatorId                     int64
creatorName                  object
creatorUrl                   object
currency                     object
currency_symbol              object
currency_trailing_code         bool
current_currency             object
deadline                      int64
description                  object
disable_communication          bool
fx_rate                     float64
goal                        float64
id                            int64
is_starrable                   bool
launched_at                   int64
launched_at_formatted        object
link                        

to review

In [ ]:
### Review DTypes

df.dtypes

# boolean columns death_event, smoking, sex, high_blood_pressure, diabetes, anaemia

#data = data.astype({'death_event':'boolean', 'sex':'boolean', 'smoking':'boolean', 'high_blood_pressure':'boolean','diabetes':'boolean','anaemia':'boolean','age':'int64'})

df.dtypes

### Null Values

In [70]:
df.isna().any()

backers_count               False
blurb                       False
categoryId                  False
categoryName                False
categorySlug                False
converted_pledged_amount    False
country                     False
country_displayable_name    False
created_at                  False
created_at_formatted        False
creatorAvatar               False
creatorId                   False
creatorName                 False
creatorUrl                  False
currency                    False
currency_symbol             False
currency_trailing_code      False
current_currency            False
deadline                    False
description                 False
disable_communication       False
fx_rate                     False
goal                        False
id                          False
is_starrable                False
launched_at                 False
launched_at_formatted       False
link                        False
locationId                  False
locationName  

No null values present in this dataset

### Drop Duplicates

In [71]:
df.shape

(17304, 47)

In [72]:
df = df.drop_duplicates()

In [73]:
df.shape

(17195, 47)

xxx duplicates have been dropped

### deal with duplicate listings at a different time

to do

### Data Values

In [74]:
for col in data:
    print(data[col].value_counts(), '\n')

3      3
77     1
61     1
1      1
24     1
151    1
5      1
39     1
145    1
305    1
Name: backers_count, dtype: int64 

Fruit-First Hydrating Freezies                                                                                                          1
Free Food Pantry, "Pay What You Can" Thrift Store and Community Hub for Arts, Education, and Empowerment                                1
Our Kickstarter will help secure a lease on a commercial kitchen and launch our full scale catering & events company in Brooklyn NY.    1
Gourmet coffee, donuts and bagels shop growing to service more of Colchester Country!                                                   1
2 Italian butchers and farmers transplanted in Vermont bring you cured meats coming from animals raised using sustainable practices     1
We want to bring quick, quality and convenient, drive thru coffee to the city of Boerne.                                                1
Un huerto Urbano Biointensivo para la producci

## Explore Cleaned Data

### Check for Correlation

In [ ]:
round(data.corr()['death_event'].sort_values(ascending=False),2)

# create correlation heatmap 

mask = np.zeros_like(data.corr())

mask[np.triu_indices_from(mask)] = True

fig, ax = plt.subplots(figsize=(10, 8))
ax = sns.heatmap(data.corr(), mask=mask, vmin=-1, vmax=1, annot=True, cmap='coolwarm')

plt.show()